<p>It is advisable that you read our introductory documentation webpage before moving on with understading the code. As it would help you understand the problem better.</p>
<p>You can check it out <a href="https://hasocfire.github.io/hasoc/2021/ichcl/index.html">here</a></p>

### Importing Libraries and initializing stopwords and stemmer

In [3]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import os, torch
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# import stemmer as hindi_stemmer
%config Completer.use_jedi = False

In [2]:
df = pd.read_csv("processed_data.csv")
df["label"].tolist()

['HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'NONE',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'HOF',
 'NONE',
 'HOF',
 'NONE',
 'NONE',
 'HOF',
 'NONE',
 'HOF',
 'HOF',
 'NONE',
 'HOF',


<p>Making a list of english and hindi stopwords. <br>The enlgish stopwords are retrieved from NLTK library as well. <br>And the hindi stopwords are retrieved from a data set on Mendeley Data. To read about how the authors compiled the list, you can check their <a href = "https://arxiv.org/ftp/arxiv/papers/2002/2002.00171.pdf" > publicaion </a> </p>
<p>Initializing an english SnowballStemmer using the NLTK library. <br>And the hindi stemmer used was produced by students of Banasthali University. You can check out their <a href="https://arxiv.org/ftp/arxiv/papers/1305/1305.6211.pdf">publication</a></p>

In [2]:
english_stopwords = stopwords.words("english")
with open('final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

## Reading Data

<p>Initializing a list of various directories that data is stored in using the glob Library.</p>

In [51]:
train_directories = []
for i in glob("data/train/*/"):
    for j in glob(i+'*/'):
        train_directories.append(j)

In [40]:
test_directories = []
for i in glob("subtask2_test/*/"):
    for j in glob(i+'*/'):
        test_directories.append(j)

In [41]:
test_directories

['subtask2_test/covid crisis/1396844054818680835/',
 'subtask2_test/covid crisis/1398274810652753924/',
 'subtask2_test/covid crisis/1395277604782022659/',
 'subtask2_test/covid crisis/1385891002209038343/',
 'subtask2_test/religious controversies/1393528907102515200/',
 'subtask2_test/religious controversies/1392808101007478794/',
 'subtask2_test/wuhan virus/1392426204775391233/',
 'subtask2_test/casteism/1394882097559400449/',
 'subtask2_test/casteism/1395755293213003779/',
 'subtask2_test/indian politics/1390362987995623428/',
 'subtask2_test/indian politics/1395657670762405888/',
 'subtask2_test/indian politics/1396686927227588609/',
 'subtask2_test/israel/1392413634773409792/',
 'subtask2_test/israel/1392157687723347968/',
 'subtask2_test/bantwitter/1397223849830359052/',
 'subtask2_test/bantwitter/1397136107108663298/']

<p>Reading tree structured data from the directories from the .json files</p>

In [52]:
data = []
for i in train_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))
labels = []
for i in train_directories:
    with open(i+'labels.json', encoding='utf-8') as f:
        labels.append(json.load(f))

In [42]:
test_data = []
for i in test_directories:
    print(i)
    with open(i+'data.json', encoding='utf-8') as f:
        test_data.append(json.load(f))

subtask2_test/covid crisis/1396844054818680835/
subtask2_test/covid crisis/1398274810652753924/
subtask2_test/covid crisis/1395277604782022659/
subtask2_test/covid crisis/1385891002209038343/
subtask2_test/religious controversies/1393528907102515200/
subtask2_test/religious controversies/1392808101007478794/
subtask2_test/wuhan virus/1392426204775391233/
subtask2_test/casteism/1394882097559400449/
subtask2_test/casteism/1395755293213003779/
subtask2_test/indian politics/1390362987995623428/
subtask2_test/indian politics/1395657670762405888/
subtask2_test/indian politics/1396686927227588609/
subtask2_test/israel/1392413634773409792/
subtask2_test/israel/1392157687723347968/
subtask2_test/bantwitter/1397223849830359052/
subtask2_test/bantwitter/1397136107108663298/


</p>Defining 2 functions that will turn the data from a tree structure to a flat structure.</p>
<ul>
    <li>tr_flatten: This is to flat the train data. It takes two variables as function parameters. First one is the tweet data and second one is labels. It'll create a list of json structures like following:
        <ul>
            <li> for source tweet: It'll create json with tweet_id, tweet text and label. </li>
            <li> for comment: It'll create json with tweet_id, label and for the text part it'll append the comment after the source tweet. This is a basic technique to provide context of source tweet. </li>
            <li> for reply: It'll create json with tweet_id, label and for the text part it'll append the reply after the comment after the source tweet. So the text here will look like "source tweet-comment-reply"</li>
        </ul>
    </li>
    <li>te_flatten: This is to flat the test data. It works similarly like tr_flatten but without the labels file, as labels won't be available for test set. It'll be used once the test data is available</li>
</ul>

In [43]:
def tr_flatten(d,l):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
        'label':l[d['tweet_id']]
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] +' [SENSEP] '+i['tweet'], #flattening comments(appending one after the other)
                'label':l[i['tweet_id']]
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] +' [SENSEP] '+ i['tweet'] +' [SENSEP] '+ j['tweet'], #flattening replies
                        'label':l[j['tweet_id']]
                    })
    return flat_text

def te_flatten(d):
    flat_text = []
    flat_text.append({
        'tweet_id':d['tweet_id'],
        'text':d['tweet'],
    })

    for i in d['comments']:
            flat_text.append({
                'tweet_id':i['tweet_id'],
                'text':flat_text[0]['text'] + ' [SENSEP] ' + i['tweet'],
            })
            if 'replies' in i.keys():
                for j in i['replies']:
                    flat_text.append({
                        'tweet_id':j['tweet_id'],
                        'text':flat_text[0]['text'] + ' [SENSEP] ' + i['tweet'] + ' [SENSEP] ' + j['tweet'],
                    })
    return flat_text

<p>This cell will run both the flatten functions. Again, you can skip the test part if it is not available. The train_len variable will be used later on for splitting the data.</p>

In [53]:
## train
data_label = []
#for train
for i in range(len(labels)):
    for j in tr_flatten(data[i], labels[i]):
        data_label.append(j)

train_len = len(data_label)

In [55]:
data_label

[{'tweet_id': '1392333796901408771',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael',
  'label': 'HOF'},
 {'tweet_id': '1392334007161868294',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP #IndiaStandsWithIsrael',
  'label': 'HOF'},
 {'tweet_id': '1392336707308060672',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP 🤣🤣',
  'label': 'HOF'},
 {'tweet_id': '1392347106208206849',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP Best😂😂',
  'label': 'HOF'},
 {'tweet_id': '1392350511270010882',
  'text': 'Madarchomd Palestine...

In [44]:
## test
test_data_flat = []
for i in range(len(test_data)):
    for j in te_flatten(test_data[i]):
        test_data_flat.append(j)

In [49]:
test_data_flat

[{'tweet_id': '1396844054818680835',
  'text': 'Bhadva Ramdev was brought to a debate about Allopathy to sit with doctors &amp; discuss his statement\nThe debate with sponsored by Coronil - https://t.co/qOP3Ucnx0s'},
 {'tweet_id': '1396844158283776004',
  'text': 'Bhadva Ramdev was brought to a debate about Allopathy to sit with doctors &amp; discuss his statement\nThe debate with sponsored by Coronil - https://t.co/qOP3Ucnx0s [SENSEP] @kunalkamra88 बाबा का विरोध करने से पहले इनके कोरोनील के पीछे जो साइंस है उसे आप यहा आसानी से समझ सकते हैं!\n\nजरूर देखिए 👇\n\nhttps://t.co/sEFSOxo0Wd'},
 {'tweet_id': '1397043581446098945',
  'text': 'Bhadva Ramdev was brought to a debate about Allopathy to sit with doctors &amp; discuss his statement\nThe debate with sponsored by Coronil - https://t.co/qOP3Ucnx0s [SENSEP] @kunalkamra88 बाबा का विरोध करने से पहले इनके कोरोनील के पीछे जो साइंस है उसे आप यहा आसानी से समझ सकते हैं!\n\nजरूर देखिए 👇\n\nhttps://t.co/sEFSOxo0Wd [SENSEP] @anupagrawal23 @kunalka

In [54]:
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)

In [47]:
test_df = pd.DataFrame(test_data_flat, columns = test_data_flat[0].keys(), index = None)

In [48]:
test_df

,tweet_id,text
0,1396844054818680835,Bhadva Ramdev was brought to a debate about Al...
1,1396844158283776004,Bhadva Ramdev was brought to a debate about Al...
2,1397043581446098945,Bhadva Ramdev was brought to a debate about Al...
3,1398265913749635073,Bhadva Ramdev was brought to a debate about Al...
4,1396852220268716032,Bhadva Ramdev was brought to a debate about Al...
...,...,...
1343,1397189986437537803,Crocodile is busy in diverting the issue to in...
1344,1397209328013164548,Crocodile is busy in diverting the issue to in...
1345,1397217889237745666,Crocodile is busy in diverting the issue to in...
1346,1397224681229504520,Crocodile is busy in diverting the issue to in...


In [60]:
df["text"][2]

'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP 🤣🤣'

In [61]:
data_label

[{'tweet_id': '1392333796901408771',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael',
  'label': 'HOF'},
 {'tweet_id': '1392334007161868294',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP #IndiaStandsWithIsrael',
  'label': 'HOF'},
 {'tweet_id': '1392336707308060672',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP 🤣🤣',
  'label': 'HOF'},
 {'tweet_id': '1392347106208206849',
  'text': 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP Best😂😂',
  'label': 'HOF'},
 {'tweet_id': '1392350511270010882',
  'text': 'Madarchomd Palestine...

In [123]:
df['label'].value_counts()

NONE    2899
HOF     2841
Name: label, dtype: int64

In [64]:
tweets = df.text
# test_tweets = test_df.text
y = df.label

In [53]:
tweets.tolist()[120:]

['israel मुर्दबाद inc israel मुर्दबाद हक़ के रास्ते पर अगर सारी कायनात भी तुम्हारे खिलाफ हो तो भी तुम यक़ीन रखो ☝ अल्लाह तुम्हारे साथ है',
 'israel मुर्दबाद inc मुर्दबाद मुर्दबाद',
 'israel मुर्दबाद inc इजराइल मुरदाबाद था मुरदाबाद है और मुरदाबाद रहेगा इजराइल अतकवादी बैकयूट इस्रायल',
 'israel मुर्दबाद inc israel मुर्दबाद tha hai or कयामत tak रहेगा',
 'israel मुर्दबाद inc why israel मुर्दबाद focus on india leave the internal matters of israel and palestine to their individual governments who will do a good job if the world stops interfering and instigating hatred between the two nations',
 'israel मुर्दबाद inc why israel मुर्दबाद focus on india leave the internal matters of israel and palestine to their individual governments who will do a good job if the world stops interfering and instigating hatred between the two nations inc saudi sent the oxygen to india something which every indian highly appreciates but what context does the oxygen have to do with the primary tweet or to my tweet'

In [171]:
from ai4bharat.transliteration import XlitEngine
e = XlitEngine("hi")
out = e.translit_word("कम्प्यूटर", topk=5, beam_width=10)
print(out['hi'][0])

Loading hi...
कम्प्यूटर


In [68]:
tweets.tolist()

['Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael',
 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP #IndiaStandsWithIsrael',
 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP 🤣🤣',
 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP Best😂😂',
 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsrael [SENSEP] @GareeboOP Best😂😂 [SENSEP] @HinduBee 😂😂',
 'Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.\n#IndiaStandsWithIsr

## Preprocessing and featuring the raw text

In [207]:
nltk.download('words', download_dir="/media/nas_mount/Zaki/nltk_data/")

[nltk_data] Downloading package words to
[nltk_data]     /media/nas_mount/Zaki/nltk_data/...
[nltk_data]   Unzipping corpora/words.zip.


True

In [103]:
import spacy
from ai4bharat.transliteration import XlitEngine
e = XlitEngine("hi")
nlp = spacy.load('en_core_web_sm')

Loading hi...


In [104]:
import string
import emoji
import nltk

nltk.data.path.append('/media/nas_mount/Zaki/nltk_data/')
regex_for_emojis="['\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF']"
# regex_for_english_hindi_emojis="[^a-zA-Z\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
import nltk, spacy
en_words = list(set(nltk.corpus.words.words() + list(nlp.vocab.strings)))

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

def clean_tweet(tweet):
#     tweet = re.sub(r"@[A-Za-z0-9_-]+",' ', tweet)
#     tweet = re.sub(r"#[A-Za-z0-9_-]+",' ', tweet)
    tweet = re.sub(r"#(\w+)", ' ', tweet)
    tweet = re.sub(r"@(\w+)", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(r"http?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(r"www?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub( r'(\w)([,.!-/+?!^$<>])', r'\1 \2', tweet )
    tweet = re.sub( r'([,.!-/+?!^$<>])(\w)', r'\1 \2', tweet )
    tweet = remove_emoji(tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
#     tweet = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', tweet)
    tweet = tweet.lower()
    tokens = []
    for token in tweet.split():
#         print(token, lang)
        if (token == '[sensep]'):
            tokens.append('[SENSEP]')
            continue
        if token not in en_words and token not in string.punctuation:
            out = e.translit_word(token, topk=1, beam_width=10)
            tokens.append(out['hi'][0])
#             print(token)
        else:
            tokens.append(token)
    return " ".join(tokens)

In [222]:
from tqdm import tqdm
cleaned_tweets = []
for tweet in tqdm(tweets[:50]):
    print(tweet)
    cleaned_tweets.append(clean_tweet(tweet))
# cleaned_tweets_te = [clean_tweet(tweet) for tweet in test_tweets]

  2%|▏         | 1/50 [00:00<00:08,  6.04it/s]

Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.
#IndiaStandsWithIsrael
Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.
#IndiaStandsWithIsrael [SENSEP] @GareeboOP #IndiaStandsWithIsrael


  6%|▌         | 3/50 [00:00<00:07,  6.01it/s]

Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.
#IndiaStandsWithIsrael [SENSEP] @GareeboOP 🤣🤣
Madarchomd Palestine...If the swara bhaskar is supporting you.. then you really deserve what Israel doing with you.
#IndiaStandsWithIsrael [SENSEP] @GareeboOP Best😂😂


 10%|█         | 5/50 [00:00<00:07,  5.92it/s]

...

 96%|█████████▌| 48/50 [00:32<00:01,  1.45it/s]

You Fucked Them &amp; That Was The Worst Thing You Ever Did !!! #Isreal https://t.co/dOdnN377u5 [SENSEP] @thakkar_sameet Isko kehtein hai "Ab Tandav hoga".
Next time yaad rakhna ki Maulana ki aukaat nahi hai Aisa karne ki


 98%|█████████▊| 49/50 [00:35<00:01,  1.34s/it]

You Fucked Them &amp; That Was The Worst Thing You Ever Did !!! #Isreal https://t.co/dOdnN377u5 [SENSEP] @thakkar_sameet This is a fake title


100%|██████████| 50/50 [00:35<00:00,  1.41it/s]


In [6]:
def get_binary_labels(labels):
    for i, label in enumerate(labels):
        if(label=="HOF"):
            labels[i] = 1
        else:
            labels[i] = 0

    return np.array(labels) 

## loading processed dataset

In [7]:
data = pd.read_csv("processed_data_punc.csv")
tweets = data["tweet"].tolist()
labels = data["label"].tolist()

In [8]:
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit
# stratSplit = StratifiedShuffleSplit(labels, n_iter=1, test_size=0.5, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(tweets, labels)
tweets = np.array(tweets)
labels = get_binary_labels(labels)

for train_index, test_index in sss.split(tweets, labels):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = tweets[train_index], tweets[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

TRAIN: [4464 1219 4378 ... 4281 4454  482] TEST: [5223 4077 3257 ... 3994 1494 3310]


In [14]:
np.save("x_test", x_test)

In [27]:
from transformers import AutoTokenizer
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, additional_special_tokens=["[SENSEP]"])

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [25]:
tokenizer.special_tokens_map

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '<unk>',
 'sep_token': '[SEP]',
 'pad_token': '<pad>',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'additional_special_tokens': "['[SENSEP]']"}

In [6]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
name = "xlm-roberta-base" #"ai4bharat/indic-bert"
model = AutoModelForSequenceClassification.from_pretrained(name, cache_dir="/media/nas_mount/Zaki/models/")
tokenizer = AutoTokenizer.from_pretrained(name, additional_special_tokens=["[SENSEP]"])
train_encodings = tokenizer(list(x_train), return_tensors='pt', padding=True, max_length=100, truncation=True)
test_encodings = tokenizer(list(x_test), return_tensors='pt', padding=True, max_length=100, truncation=True)

import torch

class HasocDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        # print(item)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HasocDataset(train_encodings, y_train)
test_dataset = HasocDataset(test_encodings, y_test)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [8]:
len(test_dataset)

1148

In [9]:
from transformers import  Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='/media/nas_mount/Zaki/hasoc/results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='/media/nas_mount/Zaki/hasoc/logs',
    learning_rate=2e-5,
    do_eval=True,
    evaluation_strategy="epoch",
)

#     n_gpu=1,
#     sharded_ddp="zero_dp_3",
    
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,   
#     optimizers=(optimizer,None)
)

In [10]:
trainer.train()

/home/zaki/.conda/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.659012,0.605625,0.679443,0.678031,0.681504,0.678825


/home/zaki/.conda/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


TrainOutput(global_step=574, training_loss=0.6516763707071231)

# test dataset predictions

In [131]:
test_csv = pd.read_csv("processed_test.csv")
test_tweets = test_csv["text"].tolist()
test_tweets

['भडवा रामदेव was brought to a debate about allopathy to sit with doctors & एम्प; discuss his statement the debate with sponsored by कोरोनिल -',
 'भडवा रामदेव was brought to a debate about allopathy to sit with doctors & एम्प; discuss his statement the debate with sponsored by कोरोनिल - [SENSEP] बाबा का विरोध करने से पहले इनके कोरोनील के पीछे जो साइंस है उसे आप यहा आसानी से समझ सकते हैं! जरूर देखिए',
 'भडवा रामदेव was brought to a debate about allopathy to sit with doctors & एम्प; discuss his statement the debate with sponsored by कोरोनिल - [SENSEP] बाबा का विरोध करने से पहले इनके कोरोनील के पीछे जो साइंस है उसे आप यहा आसानी से समझ सकते हैं! जरूर देखिए [SENSEP] the only evidence this fraud can provide is a lie !',
 'भडवा रामदेव was brought to a debate about allopathy to sit with doctors & एम्प; discuss his statement the debate with sponsored by कोरोनिल - [SENSEP] बाबा का विरोध करने से पहले इनके कोरोनील के पीछे जो साइंस है उसे आप यहा आसानी से समझ सकते हैं! जरूर देखिए [SENSEP] कोरोनिल और

In [141]:
# len(test_encodings['input_ids'])

1348

In [144]:
test_encodings = tokenizer(list(test_tweets), return_tensors='pt', padding=True, max_length=64, truncation=True)

class HasocDataset1(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
        # print(item)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])
    
test1_dataset = HasocDataset1(test_encodings)

In [1]:
trainer.predict(test1_dataset)

In [24]:
predictions = trainer.predict(test_dataset)
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)
# for p in predictions:
    

/home/zaki/.conda/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


In [33]:
test_labels = []
for i in range(len(test_dataset)):
    test_labels.append(test_dataset[i]['labels'].item())
test_labels

/home/zaki/.conda/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()


[1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [92]:
precision_recall_fscore_support(test_labels, preds, average='macro')

/home/zaki/.conda/envs/thesis/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.25261324041811845, 0.5, 0.3356481481481481, None)

In [90]:
from tqdm import tqdm
def encode_data(test_sentences):

    test_input_ids = []
    test_attention_masks = []

    # For every sentence...
    for i, sent in enumerate(tqdm(test_sentences)):
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 512,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                            truncation=True
                       )

        test_input_ids.append(encoded_dict['input_ids'])
        test_attention_masks.append(encoded_dict['attention_mask'])
        
    return test_input_ids, test_attention_masks


def get_predictions(trained_model, test_input_ids, test_attention_masks):
    predictions = []
    for i in tqdm(range(len(test_input_ids))):
        b_input_ids = torch.cat([test_input_ids[i]], dim=0).cuda()
        b_attention_masks = torch.cat([test_attention_masks[i]], dim=0).cuda()

        with torch.no_grad():
            (logits) = trained_model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_attention_masks,
                              labels=None)
        b_input_ids = b_input_ids.detach()
        b_attention_masks = b_attention_masks.detach()
        logits = torch.nn.functional.softmax(logits[0]).cpu()
        predictions+=logits
    return predictions

model.eval()
test_input_ids, test_attention_masks = encode_data(x_test)
predictions = get_predictions(model, test_input_ids, test_attention_masks)

  0%|          | 0/1148 [00:00<?, ?it/s]/home/zaki/.conda/envs/thesis/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1148/1148 [00:24<00:00, 46.87it/s]


In [91]:
preds = [np.argmax(p[i].cpu()) for p in predictions]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


## Predicting test data and making a sample submission file

<p>This part will be used to read and make predictions on the test data once the it is made available. When it is available, make a directory in data directory as 'test' and copy the story direcotries into the test directory.</p>

In [110]:
test_directories = []
for i in glob("subtask2_test/*/"):
    for j in glob(i+'*/'):
        test_directories.append(j)

In [111]:
test_directories

['subtask2_test/covid crisis/1396844054818680835/',
 'subtask2_test/covid crisis/1398274810652753924/',
 'subtask2_test/covid crisis/1395277604782022659/',
 'subtask2_test/covid crisis/1385891002209038343/',
 'subtask2_test/religious controversies/1393528907102515200/',
 'subtask2_test/religious controversies/1392808101007478794/',
 'subtask2_test/wuhan virus/1392426204775391233/',
 'subtask2_test/casteism/1394882097559400449/',
 'subtask2_test/casteism/1395755293213003779/',
 'subtask2_test/indian politics/1390362987995623428/',
 'subtask2_test/indian politics/1395657670762405888/',
 'subtask2_test/indian politics/1396686927227588609/',
 'subtask2_test/israel/1392413634773409792/',
 'subtask2_test/israel/1392157687723347968/',
 'subtask2_test/bantwitter/1397223849830359052/',
 'subtask2_test/bantwitter/1397136107108663298/']

<p>The test directories do not contain labels.json file so labels list is not initialized for test data.</p>

In [112]:
test_data = []
for i in test_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))

<p>Flattening the test data.</p>

In [113]:
test_tweetid_data = []
#for test
for i in range(len(labels), len(data)):
    for j in te_flatten(data[i]):
        test_tweetid_data.append(j)

In [114]:
test_df = pd.DataFrame(test_tweetid_data, columns = test_tweetid_data[0].keys(), index = None)

In [115]:
test_df.head()

,tweet_id,text
0,1396844054818680835,Bhadva Ramdev was brought to a debate about Al...
1,1396844158283776004,Bhadva Ramdev was brought to a debate about Al...
2,1397043581446098945,Bhadva Ramdev was brought to a debate about Al...
3,1398265913749635073,Bhadva Ramdev was brought to a debate about Al...
4,1396852220268716032,Bhadva Ramdev was brought to a debate about Al...


In [116]:
test_tweets = test_df.text
tweet_ids = test_df.tweet_id

In [117]:
cleaned_test = [clean_tweet(tweet) for tweet in test_tweets]

In [118]:
X_test = vectorizer.transform(cleaned_test)
X_test = X_test.todense()

In [119]:
submission_prediction = classifier.predict(X_test)
submission = {'tweet_id': tweet_ids, 'label':submission_prediction}
submission = pd.DataFrame(submission)

In [120]:
submission.to_csv('data/sample_submission.csv', index = False)